In [1]:
import numpy as np
import csv,random
from __future__ import division
import matplotlib.pyplot as plt
import scipy as sp
import scipy.stats
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix

def cohen_kappa_score(y1, y2, labels=None, weights=None, sample_weight=None):
   
    confusion = confusion_matrix(y1, y2, labels=labels)
    n_classes = confusion.shape[0]
    sum0 = np.sum(confusion, axis=0)
    sum1 = np.sum(confusion, axis=1)
    expected = np.outer(sum0, sum1) / np.sum(sum0)

    if weights is None:
        w_mat = np.ones([n_classes, n_classes], dtype=np.int)
        w_mat.flat[:: n_classes + 1] = 0
    elif weights == "linear" or weights == "quadratic":
        w_mat = np.zeros([n_classes, n_classes], dtype=np.int)
        w_mat += np.arange(n_classes)
        if weights == "linear":
            w_mat = np.abs(w_mat - w_mat.T)
        else:
            w_mat = (w_mat - w_mat.T) ** 2
    else:
        raise ValueError("Unknown kappa weighting type.")

    k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
    return 1 - k


def LinePlot(score_true,score_surrogate,error_surrogate,num_w,label_group):
    
    #order = sorted(range(len(score_true)), key=lambda k: score_true[k])
    #score_true = [ score_true[i] for i in order]
    #score_surrogate = [ score_surrogate[i] for i in order]
    #error_surrogate = [error_surrogate[i] for i in order]
    plt.plot(score_true,'red', marker='^',label=label_group)
    plt.plot(score_surrogate,'blue',marker='*',label='Surrogate'+label_group)
    plt.errorbar(range(num_w),score_surrogate, error_surrogate, linestyle='None', marker='^')

    plt.ylabel('Score')
    plt.xlabel('Worker')
    plt.legend()
    plt.show()


def BarPlot(score_true,score_surrogate,error_surrogate,num_w,label_group1,label_group2):

    order = sorted(range(len(score_surrogate)), key=lambda k: score_true[k])
    score_true = [ score_true[i] for i in order]
    score_surrogate = [ score_surrogate[i] for i in order]
    error_surrogate = [error_surrogate[i] for i in order]

    bar_width = 0.4
    r1 = range(num_w)
    r2 = [x + bar_width for x in r1]
    plt.bar(r1,score_true, width = bar_width, color='red', edgecolor = 'black', label=label_group1)
    plt.bar(r2,score_surrogate, width = bar_width, color='cyan', edgecolor = 'black', yerr = error_surrogate, label=label_group2)

#plt.bar(score_surrogate,'blue',marker='*',label='Surrogate Brier')
#plt.errorbar(range(num_w),score_surrogate, error_surrogate, linestyle='None', marker='^')
    plt.xticks([r + bar_width for r in range(num_w)],[''])#, ['cond_A', 'cond_B', 'cond_C'])
    plt.ylabel('Average score across all tasks')
    plt.xlabel('Participants (ordered by their true scores)')
    plt.legend(loc=2)
    plt.xlim([0,num_w])
    plt.show()

def machine_error(P0,p0,p1,train_data,machine_predict):
    num_task = len(train_data)
    q1 = []
    q2 = []
    for n in range(num_task):
        q1.append(float(machine_predict[n]))
        if float(train_data[n]) == float(machine_predict[n]):
            if float(machine_predict[n]) == 1:
                q2.append(1)
            else:
                q2.append(0)
        else:
            q2.append(0)
    c1 = np.mean(q1)
    c2 = np.mean(q2)
    #print c1,c2,p1,p0
    p0_m = (c1*(1-p1)-c2)/(P0*(1-p1-p0))
    #p1_m = 1 - (c1-P0*p0_m)/(1-P0)
    p1_m = 1 - (c1*p0-c2)/((1-P0)*(p1+p0-1))
    return p0_m,p1_m
    
def error_rate(P0,train_data):

	P1 = 1-P0
	#f_r = open(filename,'r')
	#task id, label 1, label 2, label 3
	#each task is assigned to three agents; if not, we will simply skip this line of data
	p1 = []
	p2 = []
	p3 = []

	for ans in train_data:
		p1.append(float(ans[0]))
		if float(ans[0]) == float(ans[1]):
			p2.append(1)
			if (float(ans[0]) == float(ans[2])) & (float(ans[2]) == 1):
				p3.append(1)
			else:
				p3.append(0)
		else:        
			p2.append(0)
			p3.append(0)            
            

	rho = P0/P1
	P0_norm = (1-np.mean(p1))/P1
	q_norm = np.mean(p2)/P1
	q3 = np.mean(p3)

	a = 2*rho**2+2*rho
	b = -4*rho**2-4*rho+4*rho*P0_norm
	c = (rho+1-P0_norm)**2+(P0_norm-rho)**2+rho-q_norm

	#initialization
	p0_est = 0.5
	p1_est = 0.5
	p0_est1 = 0.5
	p1_est1 = 0.5
	p0_est2 = 0.5
	p1_est2 = 0.5

	if b**2-4*a*c >= 0:
		p0_est1 = (-b-(b**2-4*a*c)**0.5)/(2*a)
		p1_est1 = P0_norm - rho*(1-p0_est1)
		p0_est2 = (-b+(b**2-4*a*c)**0.5)/(2*a)
		p1_est2 = P0_norm - rho*(1-p0_est2)



	err_root1 = np.abs(P0*(p0_est1)**3+P1*(1-p1_est1)**3-q3)
	err_root2 = np.abs(P0*(p0_est2)**3+P1*(1-p1_est2)**3-q3)

	if err_root1 < err_root2:
		p0_est = p0_est1
		p1_est = p1_est1
	else:
		p0_est = p0_est2
		p1_est = p1_est2
        
	if p0_est+p1_est >= 1:
		ind = -1
	else:
		ind = 1
	p1_est = 1 - (np.mean(p1)-P0*p0_est)/(1-P0) 

	return p0_est,p1_est,ind


def error_rate_priorfree(P0,train_data):

	P1 = 1-P0
	#f_r = open(filename,'r')
	#task id, label 1, label 2, label 3
	#each task is assigned to three agents; if not, we will simply skip this line of data
	p1 = []
	p2 = []
	p21 = []
	p3 = []

	for ans in train_data:
		p1.append(float(ans[0]))
		if float(ans[0]) == float(ans[1]):
            
			if float(ans[0]) == 1:
				p21.append(1)
			else:
				p21.append(0)
                
			p2.append(1)
            
			if (float(ans[0]) == float(ans[2])) & (float(ans[2]) == 1):
				p3.append(1)
			else:
				p3.append(0)
		else:        
			p2.append(0)
			p21.append(0)
			p3.append(0)            
            
	c1 = np.mean(p1)
	c2 = np.mean(p21)
	c3 = np.mean(p3)
	a = (c3-c1*c2)/(c2-c1**2)
	b = (c1*c3-c2**2)/(c2-c1**2)
	#print c1,c2,c3    
	if a**2-4*b >= 0:
		p0_est1 = (a-(a**2-4*b)**0.5)/2
		p1_est1 = 1-(a-p0_est1)
		p0_est2 = (a+(a**2-4*b)**0.5)/2
		p1_est2 = 1-(a-p0_est2)
	else:
		p0_est1 = 0.5
		p1_est1 = 0.5
		p0_est2 = 0.5
		p1_est2 = 0.5  

	gap_L = abs(p0_est1-c1)
	gap_H = abs(a-p0_est1-c1)
	epsilon = 0.1
	#print 'gaps are:~',gap_L,gap_H
    
	if gap_L >= gap_H + epsilon:
		if P0 > 0.5:
			p0_est = p0_est2
			p1_est = p1_est2
		else:
			p0_est = p0_est1
			p1_est = p1_est1
	else:
		if P0 > 0.5:
			p0_est = p0_est1
			p1_est = p1_est1
		else:
			p0_est = p0_est2
			p1_est = p1_est2
 


	P0_est1 = (c1-(1-p1_est1))/(p0_est1 - (1-p1_est1))
	P0_est2 = (c1-(1-p1_est2))/(p0_est2 - (1-p1_est2))
	if p0_est1 + p1_est1 > p0_est2 + p1_est2:        
		epsilon = -1*epsilon
	else:
		epsilon = epsilon

	if P0 > 0.5:
		if P0_est1+epsilon > 0.5:
			p0_est = p0_est1
			p1_est = p1_est1
		else:
			p0_est = p0_est2
			p1_est = p1_est2
	else:
		if P0_est1-epsilon > 0.5:
			p0_est = p0_est2
			p1_est = p1_est2
		else:
			p0_est = p0_est1
			p1_est = p1_est1 

	P0_est = (c1-(1-p1_est))/(p0_est - (1-p1_est))
	if p0_est+p1_est > 1:
	#if abs(P0_est1-P0_est2)<0.1:
		ind = -1
	else:
		ind = 1
        
	#p1_est = 1 - (c1-P0_est*p0_est)/(1-P0_est) 
	return p0_est,p1_est,P0_est,ind #c1


def perform_eval(e1,e0,num_q,data_sheet,data_prediction):
    
    predict_DTS = [0 for y in range(num_q)]
    predict_Maj = [0 for y in range(num_q)]
    predict_BTS = [0 for y in range(num_q)]
    cnt_DTS = 0
    cnt_Maj = 0
    cnt_BTS = 0
    truth_float = [0 for y in range(num_q)]
    for q in range(num_q):
    
        worker_ans = str(data_sheet[q])
        post0 = worker_ans.count('0')/num_w
        post1 = 1 - post0
    
        worker_pred = data_prediction[q]
        pred1 = np.mean(worker_pred)
    
        if abs(post1-e0) < abs(post1-(1-e1)):
            predict_DTS[q] = 0
        else:
            predict_DTS[q] = 1
    
        if post1 > pred1:
            predict_BTS[q] = 1
        else:
            predict_BTS[q] = 0
    
        if post0 < post1:
            predict_Maj[q] = 1
        else:
            predict_Maj[q] = 0
    
        if predict_DTS[q] == float(truth[q]):
            cnt_DTS = cnt_DTS + 1
        if predict_Maj[q] == float(truth[q]):
            cnt_Maj = cnt_Maj + 1
        if predict_BTS[q] == float(truth[q]):
            cnt_BTS = cnt_BTS + 1
    
        truth_float[q] = float(truth[q])
    
    return predict_DTS,predict_Maj,predict_BTS,cnt_DTS,cnt_Maj,cnt_BTS,truth_float

def perform_eval_hard(e1,e0,num_q,data_sheet,data_prediction,predict_MMTS):
    
    predict_DTS = [0 for y in range(num_q)]
    predict_Maj = [0 for y in range(num_q)]
    predict_BTS = [0 for y in range(num_q)]
    cnt_DTS = 0
    cnt_Maj = 0
    cnt_BTS = 0
    cnt_MMTS = 0
    
    truth_float = [0 for y in range(num_q)]
    
    for q in range(num_q):
                
        worker_ans = str(data_sheet[q])
        post0 = worker_ans.count('0')/num_w
        post1 = 1 - post0
    
        worker_pred = data_prediction[q]
        pred1 = np.mean(worker_pred)
        
        if post0 < post1:
            predict_Maj[q] = 1
        else:
            predict_Maj[q] = 0
            
        if abs(post1-e0) < abs(post1-(1-e1)):
            predict_DTS[q] = 0
        else:
            predict_DTS[q] = 1
    
        if post1 > pred1:
            predict_BTS[q] = 1
        else:
            predict_BTS[q] = 0
        
        #see whether majority voting gets it wrong. 
        if predict_Maj[q] != float(truth[q]):
            cnt_Maj = cnt_Maj + 1
            if predict_DTS[q] == float(truth[q]):
                cnt_DTS = cnt_DTS + 1
            if predict_BTS[q] == float(truth[q]):
                cnt_BTS = cnt_BTS + 1
            if predict_MMTS[q] == float(truth[q]):
                cnt_MMTS = cnt_MMTS + 1
        
        truth_float[q] = float(truth[q])
    print(cnt_DTS,cnt_BTS,cnt_MMTS,cnt_Maj)
    return cnt_DTS/cnt_Maj,cnt_BTS/cnt_Maj,cnt_MMTS/cnt_Maj,cnt_Maj